In [1]:
from premethod import *
from pegasus_fine_tune import *
from generate_pegasus import *
from combine import *
from rank import rank
from rank_training import rank_train
from p_at_1 import p_at_k
from keybart_finetune import *
from keybart_generate import get_pred_Keybart
from bart_finetune import fine_tune_bart
from bart_generate import get_pred_bart
### 

[nltk_data] Downloading package stopwords to /home/delab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-10-27 14:50:36,390 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: cuda
2022-10-27 14:50:36,391 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2022-10-27 14:50:36,564 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cuda
[nltk_data] Downloading package stopwords to /home/delab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
datadir = ['../dataset/EUR-Lex/','../dataset/Wiki500K/','../dataset/AmazonCat-13K/']
k_fold = [0,1,2,3,4]
tasks = ['test','train','valid']
models = {'pega':0,'bart':1,'kb':0}
gener = "generate_result/"
da = 2 #AmazonCat-13K

In [4]:
dataname=[datadir[da]+tasks[0]+"_labels.txt",datadir[da]+tasks[1]+"_labels.txt"]
get_all_labels(dataname,datadir[da]+"all_labels.txt")
#get_all_stemlabels(datadir[1]+'all_labels20.txt',datadir[1]+'all_stemlabels20.txt')

In [ ]:
# pegasus
if models['pega']:
        Pegasus_fine_tune(datadir[0],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json',"pegasus_save","pegasus_check")
        for i in range(2):
            get_pred_Pegasus_fast(datadir[1],gener+tasks[i]+"_pred.txt",tasks[i]+"_finetune.json","pegasus_save")
            get_combine_list(datadir[1],gener+tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
        rank_train(datadir[1],tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en")
        rank(datadir[1],tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en",tasks[0]+"_ranked_labels.txt")
        res = p_at_k(datadir[1],tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[1]+"res_pega.txt")


In [3]:
if models['bart']:
        fine_tune_bart(datadir[1],tasks[1]+'_finetune20.json',tasks[0]+'_finetune20.json','bart_save','bart_check')
        for i in range(2):
            get_pred_bart(datadir[1],gener+tasks[i]+"_pred_ba.txt",tasks[i]+"_finetune20.json","bart_save")
            bart_clean(datadir[1]+gener+tasks[i]+"_pred_ba.txt",datadir[1]+gener+tasks[i]+"_pred_ba_c.txt")
            get_combine_list(datadir[1],gener+tasks[i]+"_pred_ba.txt","all_labels20.txt",gener+tasks[i]+"_combine_labels_ba.txt")
        rank_train(datadir[1],tasks[1]+"_texts.txt",gener+tasks[1]+"_combine_labels_ba.txt",tasks[1]+"_labels20.txt","cr_en_ba")
        rank(datadir[1],tasks[0]+"_texts20.txt",gener+tasks[0]+"_combine_labels_ba.txt","cr_en_ba",gener+tasks[0]+"_ranked_labels_ba.txt")
        res = p_at_k(datadir[1],tasks[0]+"_labels20.txt",gener+tasks[0]+"_combine_labels_ba.txt",datadir[1]+"ba_res.txt")

p_at_k:

src_label: ../dataset/Wiki500K/test_labels20.txt
pred_label: ../dataset/Wiki500K/generate_result/test_combine_labels_ba.txt
p@1= 0.5776258038174952
p@3= 0.4747711884590521
p@5= 0.3861284066550985


In [ ]:
if models['kb']:    
        #kb_fine_tune(datadir[0],"kb_save","kb_check")
        fine_tune_keybart(datadir[1],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json','keybart_save','keybart_test')
        for i in range(2):
            get_pred_Keybart(datadir[0],gener+tasks[i]+"_pred_kb.txt",tasks[i]+"_finetune.json","keybart_save")
            bart_clean(datadir[0]+gener+tasks[i]+"_pred_kb.txt",datadir[0]+gener+tasks[i]+"_pred_kb_c.txt")
            get_combine_list(datadir[0],gener+tasks[i]+"_pred_kb_c.txt","all_labels_sterm.txt",gener+tasks[i]+"_combine_labels_kb.txt")

##### K则验证

In [ ]:
for j in k_fold:
    k_dir = "/K_fold/"+"K_"+str(j)+"/"
    Pegasus_fine_tune(datadir[0]+k_dir,tasks[1]+'.json',tasks[0]+'.json',"pegasus_save","pegasus_check")
    for i in range(2):
        get_pred_Pegasus(datadir[0]+k_dir,tasks[i]+"_pred.txt",tasks[i]+".json","pegasus_save")
        get_combine_list(datadir[0]+k_dir,tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
    rank_train(datadir[0]+k_dir,tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en_"+str(j))
    rank(datadir[0]+k_dir,tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en_"+str(j),tasks[0]+"_ranked_labels.txt")
    res = p_at_k(datadir[0]+k_dir,tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[0]+k_dir+"res.txt")